In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
mission_name = "Zero-20220221"

In [ ]:
%run -i Trajectory_Optimization.py Zero-new

In [ ]:
display_6DoF(out)

In [ ]:
out.loc[:,["time","heading","pitch","roll"]].to_csv("../RocketSim/bin/input/OptResult-Attitude.csv",index=None)

In [ ]:
ref = pd.read_csv("ALMA/ZERO_NOMINAL_2201.csv",skiprows=[0,1,2])

In [ ]:
df = pd.read_csv("../RocketSim/bin/output/OptResult-out.csv")

In [ ]:
plt.plot(ref["time"],ref["h"],lw=0.8)
plt.plot(out["time"],out["alt"])
plt.plot(df["time"],df["alt"])
plt.grid()

df1  = np.interp(out["time"], df["time"], df["alt"])
plt.figure()
plt.plot(out["time"], df1-out["alt"], ".--", lw=0.8)
plt.xlim([0,500])
plt.grid()

In [ ]:
plt.plot(ref["time"], ref["ha"], lw=0.8)
plt.plot(out["time"], out["ha"],".:")
plt.plot(df["time"], df["ha"])
plt.xlim([0,500])
plt.ylim([-20,None])
plt.grid()

df1  = np.interp(out["time"], df["time"], df["ha"])
plt.figure()
plt.plot(out["time"], df1-out["ha"], ".--", lw=0.8)
plt.xlim([0,500])
plt.grid()

In [ ]:
plt.plot(ref["time"], ref["hp"], lw=0.8)
plt.plot(out["time"], out["hp"],".:")
plt.plot(df["time"], df["hp"])
plt.xlim([0,500])
plt.ylim([-1000,None])
plt.grid()

df1  = np.interp(out["time"], df["time"], df["hp"])
plt.figure()
plt.plot(out["time"], df1-out["hp"], ".--", lw=0.8)
plt.xlim([0,500])
plt.grid()

In [ ]:
plt.plot(out["time"],out["vr"],".:")
plt.plot(df["time"],df["vel_ground"])
plt.xlim([0,500])
plt.grid()

df1  = np.interp(out["time"], df["time"], df["vel_ground"])
plt.figure()
plt.plot(out["time"], df1-out["vr"], ".--", lw=0.8)
plt.xlim([0,500])
plt.grid()

In [ ]:
plt.plot(ref["time"],ref["eularGDB[1]"],lw=0.8)
plt.plot(out["time"],out["pitch"],".:")
plt.plot(df["time"],df["pitch"].to_numpy('f8'),lw=0.8)
plt.grid()

df1  = np.interp(out["time"], df["time"], df["pitch"])
plt.figure()
plt.plot(out["time"], df1-out["pitch"], ".--", lw=0.8)
plt.xlim([0,500])
plt.grid()

In [ ]:
plt.plot(ref["time"],ref["eularGDB[2]"],lw=0.8)
plt.plot(out["time"],out["heading"],".:")
plt.plot(df["time"],df["heading"].to_numpy('f8'),lw=0.8)
plt.ylim([60,120])
plt.grid()

In [ ]:
plt.plot(out["time"],out[["quat_i2b_0","quat_i2b_1","quat_i2b_2","quat_i2b_3"]],".:")
plt.plot(df["time"],df[["quat_i2b_0","quat_i2b_1","quat_i2b_2","quat_i2b_3"]].to_numpy('f8'),lw=0.8)
plt.ylim([-1,1])
plt.grid()

In [ ]:
#plt.plot(ref["time"],ref["axb"])
plt.plot(out["time"],out["accel_X"],".:")
plt.plot(df["time"],df["acc_BODY_X"])
plt.grid()

df1  = np.interp(out["time"], df["time"], df["acc_BODY_X"])
plt.figure()
plt.plot(out["time"], df1-out["accel_X"], ".--", lw=0.8)
plt.xlim([0,500])
plt.ylim([-0.05,0.05])
plt.grid()

In [ ]:
plt.plot(ref["time"],ref["Fr"],lw=0.8)
plt.plot(out["time"],out["thrust"],"--")
plt.plot(df["time"],df["thrust"],"--")
plt.grid()

df1  = np.interp(out["time"], df["time"], df["thrust"])
plt.figure()
plt.plot(out["time"], df1-out["thrust"], ".--", lw=0.8)
plt.xlim([0,500])
plt.ylim([-300,300])
plt.grid()

In [ ]:
#plt.plot(ref["time"],ref["weight"])
plt.plot(out["time"],out["mass"],lw=0.8)
plt.plot(df["time"],df["total mass"],lw=0.8)
plt.xlim([0,500])
plt.grid()

df1  = np.interp(out["time"], df["time"], df["total mass"])
plt.figure()
plt.plot(out["time"], df1-out["mass"], ".--", lw=0.8)
plt.xlim([0,500])
plt.ylim([-0.1,0.1])
plt.grid()

In [ ]:
plt.plot(ref["time"],ref["q"])
plt.plot(out["time"],out["q"],".:",lw=0.8)
plt.plot(df["time"],df["q"],lw=0.8)
plt.xlim([0,500])
plt.yscale('log')
plt.ylim([1e-3,None])
plt.grid()

In [ ]:
plt.plot(ref["time"],ref["alphaq"])
plt.plot(out["time"],out["q"]*out["aoa_total"],".:",lw=0.8)
plt.plot(df["time"],df["q"]*df["aoa_alpha"],lw=0.8)
plt.xlim([0,500])
plt.grid()